<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 25, "amount": -373}, {"transaction-id": 99, "amount": -367}, {"transaction-id": 1620, "amount": -182}, {"transaction-id": 1952, "amount": -276}, {"transaction-id": 3180, "amount": -299}, {"transaction-id": 3448, "amount": -645}, {"transaction-id": 3540, "amount": -445}, {"transaction-id": 5229, "amount": -347}, {"transaction-id": 5501, "amount": -298}, {"transaction-id": 5568, "amount": -309}, {"transaction-id": 5659, "amount": -355}, {"transaction-id": 5904, "amount": -351}, {"transaction-id": 6997, "amount": -299}, {"transaction-id": 7239, "amount": -395}, {"transaction-id": 7331, "amount": -441}, {"transaction-id": 7558, "amount": -461}, {"transaction-id": 7821, "amount": -358}, {"transaction-id": 8487, "amount": -315}, {"transaction-id": 9069, "amount": -344}, {"transaction-id": 9257, "amount": -490}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 25, "amount": -373}, {"transaction-id": 99, "amount": -367}, {"transaction-id": 1620, "amount": -182}, {"transaction-id": 1952, "amount": -276}, {"transaction-id": 3180, "amount": -299}, {"transaction-id": 3448, "amount": -645}, {"transaction-id": 3540, "amount": -445}, {"transaction-id": 5229, "amount": -347}, {"transaction-id": 5501, "amount": -298}, {"transaction-id": 5568, "amount": -309}, {"transaction-id": 5659, "amount": -355}, {"transaction-id": 5904, "amount": -351}, {"transaction-id": 6997, "amount": -299}, {"transaction-id": 7239, "amount": -395}, {"transaction-id": 7331, "amount": -441}, {"transaction-id": 7558, "amount": -461}, {"transaction-id": 7821, "amount": -358}, {"transaction-id": 8487, "amount": -315}, {"transaction-id": 9069, "amount": -344}, {"transaction-id": 9257, "amount": -490}]}\n',
 '{"id": 1, "name": "Ingrid", "transactions": [{"transaction-id": 1330, "amount": 311}, {"transaction-id": 5855, 

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 25, 'amount': -373},
   {'transaction-id': 99, 'amount': -367},
   {'transaction-id': 1620, 'amount': -182},
   {'transaction-id': 1952, 'amount': -276},
   {'transaction-id': 3180, 'amount': -299},
   {'transaction-id': 3448, 'amount': -645},
   {'transaction-id': 3540, 'amount': -445},
   {'transaction-id': 5229, 'amount': -347},
   {'transaction-id': 5501, 'amount': -298},
   {'transaction-id': 5568, 'amount': -309},
   {'transaction-id': 5659, 'amount': -355},
   {'transaction-id': 5904, 'amount': -351},
   {'transaction-id': 6997, 'amount': -299},
   {'transaction-id': 7239, 'amount': -395},
   {'transaction-id': 7331, 'amount': -441},
   {'transaction-id': 7558, 'amount': -461},
   {'transaction-id': 7821, 'amount': -358},
   {'transaction-id': 8487, 'amount': -315},
   {'transaction-id': 9069, 'amount': -344},
   {'transaction-id': 9257, 'amount': -490}]},
 {'id': 1,
  'name': 'Ingrid',
  'transactions': [{'tran

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 9,
  'name': 'Alice',
  'transactions': [{'transaction-id': 10, 'amount': 974},
   {'transaction-id': 181, 'amount': 1041},
   {'transaction-id': 390, 'amount': 1056},
   {'transaction-id': 449, 'amount': 1106},
   {'transaction-id': 495, 'amount': 1078},
   {'transaction-id': 590, 'amount': 882},
   {'transaction-id': 610, 'amount': 891},
   {'transaction-id': 632, 'amount': 997},
   {'transaction-id': 922, 'amount': 947},
   {'transaction-id': 1071, 'amount': 869},
   {'transaction-id': 1190, 'amount': 1055},
   {'transaction-id': 1666, 'amount': 1037},
   {'transaction-id': 1820, 'amount': 914},
   {'transaction-id': 2066, 'amount': 1047},
   {'transaction-id': 2208, 'amount': 945},
   {'transaction-id': 2599, 'amount': 1047},
   {'transaction-id': 2910, 'amount': 990},
   {'transaction-id': 2926, 'amount': 1096},
   {'transaction-id': 3050, 'amount': 943},
   {'transaction-id': 3105, 'amount': 1007},
   {'transaction-id': 3137, 'amount': 1030},
   {'transaction-id': 3145, '

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 57},
 {'name': 'Alice', 'count': 69},
 {'name': 'Alice', 'count': 2},
 {'name': 'Alice', 'count': 2},
 {'name': 'Alice', 'count': 10})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(57, 69, 2, 2, 10)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

25.06756756756757

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 10, 'amount': 974},
  {'transaction-id': 181, 'amount': 1041},
  {'transaction-id': 390, 'amount': 1056},
  {'transaction-id': 449, 'amount': 1106},
  {'transaction-id': 495, 'amount': 1078},
  {'transaction-id': 590, 'amount': 882},
  {'transaction-id': 610, 'amount': 891},
  {'transaction-id': 632, 'amount': 997},
  {'transaction-id': 922, 'amount': 947},
  {'transaction-id': 1071, 'amount': 869},
  {'transaction-id': 1190, 'amount': 1055},
  {'transaction-id': 1666, 'amount': 1037},
  {'transaction-id': 1820, 'amount': 914},
  {'transaction-id': 2066, 'amount': 1047},
  {'transaction-id': 2208, 'amount': 945},
  {'transaction-id': 2599, 'amount': 1047},
  {'transaction-id': 2910, 'amount': 990},
  {'transaction-id': 2926, 'amount': 1096},
  {'transaction-id': 3050, 'amount': 943},
  {'transaction-id': 3105, 'amount': 1007},
  {'transaction-id': 3137, 'amount': 1030},
  {'transaction-id': 3145, 'amount': 1151},
  {'transaction-id': 3176, 'amount': 1005},
  {'tran

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 10, 'amount': 974},
 {'transaction-id': 181, 'amount': 1041},
 {'transaction-id': 390, 'amount': 1056})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(974, 1041, 1056)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

714.6587601078168

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 89), ('Alice', 89), ('Alice', 89), ('Alice', 103), ('Bob', 141), ('Bob', 143), ('Bob', 144), ('Bob', 164), ('Charlie', 78), ('Charlie', 78), ('Charlie', 79), ('Charlie', 94), ('Dan', 156), ('Dan', 157), ('Dan', 159), ('Dan', 185), ('Edith', 72), ('Edith', 72), ('Edith', 72), ('Edith', 84), ('Frank', 109), ('Frank', 112), ('Frank', 116), ('Frank', 134), ('George', 106), ('George', 107), ('George', 108), ('George', 126), ('Hannah', 144), ('Hannah', 144), ('Hannah', 144), ('Hannah', 168), ('Ingrid', 107), ('Ingrid', 108), ('Ingrid', 108), ('Ingrid', 126), ('Jerry', 119), ('Jerry', 120), ('Jerry', 120), ('Jerry', 140), ('Kevin', 176), ('Kevin', 177), ('Kevin', 180), ('Kevin', 209), ('Laura', 85), ('Laura', 85), ('Laura', 86), ('Laura', 99), ('Michael', 62), ('Michael', 64), ('Michael', 65), ('Michael', 77), ('Norbert', 47), ('Norbert', 48), ('Norbert', 48), ('Norbert', 55), ('Oliver', 132), ('Oliver', 132), ('Oliver', 132), ('Oliver', 154), ('Patricia', 96), ('Patricia', 96), ('

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 370), ('Bob', 592), ('Charlie', 329), ('Dan', 657), ('Edith', 300), ('Frank', 471), ('George', 447), ('Hannah', 600), ('Ingrid', 449), ('Jerry', 499), ('Kevin', 742), ('Laura', 355), ('Michael', 268), ('Norbert', 198), ('Oliver', 550), ('Patricia', 400), ('Quinn', 366), ('Ray', 350), ('Sarah', 750), ('Tim', 400), ('Ursula', 800), ('Victor', 599), ('Wendy', 459), ('Xavier', 600), ('Yvonne', 348), ('Zelda', 281)]
CPU times: user 182 ms, sys: 8.27 ms, total: 190 ms
Wall time: 408 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Jerry,"[{'transaction-id': 25, 'amount': -373}, {'tra..."
1,1,Ingrid,"[{'transaction-id': 1330, 'amount': 311}, {'tr..."
2,2,Patricia,"[{'transaction-id': 265, 'amount': -410}, {'tr..."
3,3,Hannah,"[{'transaction-id': 1177, 'amount': 1168}, {'t..."
4,4,Xavier,"[{'transaction-id': 213, 'amount': 2851}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 215 ms, sys: 13.1 ms, total: 228 ms
Wall time: 1.32 s


name
Alice      370
Bob        592
Charlie    329
Dan        657
Edith      300
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Jerry', 'amount': -373, 'transaction-id': 25},
 {'id': 0, 'name': 'Jerry', 'amount': -367, 'transaction-id': 99},
 {'id': 0, 'name': 'Jerry', 'amount': -182, 'transaction-id': 1620})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Jerry,-373,25
1,0,Jerry,-367,99
2,0,Jerry,-182,1620
3,0,Jerry,-276,1952
4,0,Jerry,-299,3180


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 151 ms, sys: 22.2 ms, total: 174 ms
Wall time: 922 ms


name
Alice        9275
Bob         17363
Charlie     19192
Dan         39126
Edith        7998
Frank       28606
George      15753
Hannah      32068
Ingrid      20294
Jerry       20477
Kevin       24301
Laura       14996
Michael     14954
Norbert      4154
Oliver      32428
Patricia    18461
Quinn       10808
Ray         13796
Sarah       26571
Tim         11709
Ursula      34969
Victor      20645
Wendy       18012
Xavier      29203
Yvonne      11839
Zelda        3002
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()